In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torch
import torch.nn as nn

# Define the architecture of the LSTM neural network
class LSTMNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Set the hyperparameters
input_size = 10
hidden_size = 20
num_layers = 2
output_size = 3

# Create an instance of the LSTM neural network
model = LSTMNetwork(input_size, hidden_size, num_layers, output_size)

# Define the loss function (e.g., Mean Squared Error) and optimizer (e.g., Adam)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Prepare the input and target data (assuming you already have your data)
input_data = torch.randn(100, 1, input_size)  # Shape: (batch_size, sequence_length, input_size)
target_data = torch.randn(100, output_size)

# Training loop
for epoch in range(100):
    # Forward pass
    outputs = model(input_data)
    loss = criterion(outputs, target_data)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every few epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{100}, Loss: {loss.item():.4f}')

# Save the trained model to a file
torch.save(model.state_dict(), 'trained_model.pth')

# Load the saved model from the file
loaded_model = LSTMNetwork(input_size, hidden_size, num_layers, output_size)
loaded_model.load_state_dict(torch.load('trained_model.pth'))

# Add Gaussian noise to the hidden weights of the LSTM layers in the loaded model
noise_std = 0.1  # Standard deviation of the Gaussian noise
with torch.no_grad():
    for name, param in loaded_model.named_parameters():
        if 'lstm.weight_hh' in name or 'lstm.weight_ih' in name:
            param.add_(torch.randn_like(param) * noise_std)

# Perform inference with the modified model
input_data_test = torch.randn(10, 1, input_size)  # Example test input data
output_test = loaded_model(input_data_test)
print("Output after adding noise:", output_test)

/home/rmlans/miniconda3/envs/pytorch_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 10/100, Loss: 1.0047
Epoch 20/100, Loss: 0.9180
Epoch 30/100, Loss: 0.7980
Epoch 40/100, Loss: 0.6347
Epoch 50/100, Loss: 0.4541
Epoch 60/100, Loss: 0.2663
Epoch 70/100, Loss: 0.1239
Epoch 80/100, Loss: 0.0461
Epoch 90/100, Loss: 0.0164
Epoch 100/100, Loss: 0.0054
Output after adding noise: tensor([[-1.0066,  0.3580,  0.8777],
        [-1.3239,  0.7493, -1.7620],
        [-0.6442, -0.4156,  2.0271],
        [ 3.4443,  1.8851, -0.5772],
        [-0.4363, -1.1853,  0.0615],
        [ 0.7359, -1.1814, -0.8272],
        [ 0.0572, -1.4801, -1.0127],
        [ 0.5941, -0.1645,  1.3190],
        [ 0.2780, -0.7740, -0.5391],
        [-1.6767, -0.4550,  1.0734]], grad_fn=<AddmmBackward0>)


In [5]:
# Create a new instance of the LSTM neural network
new_model = LSTMNetwork(input_size, hidden_size, num_layers, output_size)

# Load the trained weights into the model
new_model.load_state_dict(torch.load('trained_model.pth'))

# Define the testing data
input_data_test = torch.randn(10, 1, input_size)  # Example test input data
true_output_test = torch.randn(10, output_size)  # Example true output data

# Set the initial hyperparameters for the Gaussian noise
initial_noise_std = torch.tensor(0.1) # This is the value that has to be optimized given observations - can be made more complex

# Perform gradient-free optimization to tune the noise hyperparameters
optimizer = optim.SGD([initial_noise_std], lr=0.001)  # Use SGD to optimize the noise_std
criterion = nn.MSELoss()

for epoch in range(100):
    # Add Gaussian noise to the hidden weights of the LSTM layers
    with torch.no_grad():
        for name, param in model.named_parameters():
            if 'lstm.weight_hh' in name or 'lstm.weight_ih' in name:
                param.add_(torch.randn_like(param) * initial_noise_std)

    # Forward pass
    output_test = model(input_data_test)
    loss = criterion(output_test, true_output_test)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every few epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{100}, Loss: {loss.item():.4f}')

# Print the optimized noise standard deviation
optimized_noise_std = initial_noise_std.item()
print("Optimized noise standard deviation:", optimized_noise_std)

Epoch 10/100, Loss: 2.4480
Epoch 20/100, Loss: 2.2627
Epoch 30/100, Loss: 1.8238
Epoch 40/100, Loss: 1.2425
Epoch 50/100, Loss: 1.4680
Epoch 60/100, Loss: 1.7434
Epoch 70/100, Loss: 1.5112
Epoch 80/100, Loss: 1.3551
Epoch 90/100, Loss: 1.6880
Epoch 100/100, Loss: 1.9853
Optimized noise standard deviation: 0.10000000149011612
